# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [1]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 311530

2022-12-12 11:48:46.211109


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

In [13]:
!gdown 1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq -O ./data/transactions.csv
!gdown 10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE -O ./data/tr_mcc_codes.csv
!gdown 1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT -O ./data/gender_train.csv

Downloading...
From: https://drive.google.com/uc?id=1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/transactions.csv
100%|████████████████████████████████████████| 314M/314M [00:29<00:00, 10.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/tr_mcc_codes.csv
100%|██████████████████████████████████████| 14.9k/14.9k [00:00<00:00, 22.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/gender_train.csv
100%|██████████████████████████████████████| 99.9k/99.9k [00:00<00:00, 5.88MB/s]


Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [10]:
feature_list = [
    'Одежда (также нужно искать "одежды")',
    'Ценные бумаги',
    'Магазины косметики и Салоны красоты',
    'Образовательные услуги (а также школы, университеты)',
    'Прокат автомобилей',
    'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)',
    'Видеоигры (искать надо по "видеоигр")',
    'Отели',
    'Программирование',
    'Ремонт',
    'Предсказатели будущего',
    'Ювелирные изделия'
]


print ('Список категорий')
if (int(str(v)[4]) % 2) == 0:
    print(feature_list[::2])
    my_features = feature_list[::2]
else:
    print (feature_list[::-2])
    my_features = feature_list[::-2]


if (int(str(v)[5]) % 2) == 0:
    print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else:
    print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Ювелирные изделия', 'Ремонт', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Образовательные услуги (а также школы, университеты)', 'Ценные бумаги']
В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 12 признаков, то есть шесть категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [90]:
import pandas as pd

transactions = pd.read_csv('./data/transactions.csv', sep=',', nrows=1000000)
tr_mcc_codes = pd.read_csv('./data/tr_mcc_codes.csv', sep=';')
gender_train = pd.read_csv('./data/gender_train.csv', sep=',')

In [91]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [92]:
tr_mcc_codes.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [93]:
gender_train.head()

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1


In [94]:
jewelery_features = ['Ювелирные изделия']
repair_features = ['Ремонт']
hotel_features = ['Отели']
auto_features = [
    'Автозапчасти',
    'Автошины',
    'Станции техобслуживания',
    'Станции технического обслуживания',
    'СТО',
]
education_features = [
    'Образовательные услуги',
    'Школы',
    'Университеты',
]
stock_features = ['Ценные бумаги']

total_features = [
    *jewelery_features,
    *repair_features,
    *hotel_features,
    *auto_features,
    *education_features,
    *stock_features,
]

my_tr_mcc_codes = tr_mcc_codes[tr_mcc_codes.mcc_description.str.contains('|'.join(total_features))]

In [95]:
my_tr_mcc_codes

,mcc_code,mcc_description
63,5532,Автошины
64,5533,Автозапчасти и аксессуары
65,5541,Станции техобслуживания
127,6211,Ценные бумаги: брокеры/дилеры
131,7011,"Отели, мотели, базы отдыха, сервисы бронирования"
148,7531,Станции технического обслуживания для автомоби...
149,7538,СТО общего назначения
151,7629,"Ремонт бытовой техники, ремонт электроприборы"
173,8244,Школы — бизнес и секретарей
174,8299,Образовательные услуги


In [96]:
transactions = transactions.merge(my_tr_mcc_codes, on='mcc_code', how='inner')
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,mcc_description
0,39026145,2 15:53:49,5541,1010,-14643.37,NaN,Станции техобслуживания
1,39026145,7 14:23:17,5541,1010,-14643.37,NaN,Станции техобслуживания
2,39026145,14 14:29:40,5541,1010,-4873.64,NaN,Станции техобслуживания
3,39026145,14 14:30:20,5541,1010,-14643.37,NaN,Станции техобслуживания
4,39026145,16 04:27:37,5541,1010,-43391.09,NaN,Станции техобслуживания


In [97]:
# adding spending and income features
transactions['spending'] = transactions['amount'] * (transactions['amount'] < 0)
transactions['income'] = transactions['amount'] * (transactions['amount'] > 0)
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,mcc_description,spending,income
0,39026145,2 15:53:49,5541,1010,-14643.37,NaN,Станции техобслуживания,-14643.37,-0.0
1,39026145,7 14:23:17,5541,1010,-14643.37,NaN,Станции техобслуживания,-14643.37,-0.0
2,39026145,14 14:29:40,5541,1010,-4873.64,NaN,Станции техобслуживания,-4873.64,-0.0
3,39026145,14 14:30:20,5541,1010,-14643.37,NaN,Станции техобслуживания,-14643.37,-0.0
4,39026145,16 04:27:37,5541,1010,-43391.09,NaN,Станции техобслуживания,-43391.09,-0.0


In [98]:
total_categories = [
        jewelery_features,
        repair_features,
        hotel_features,
        auto_features,
        education_features,
        stock_features,
]

transactions['category'] = 'none'

for i, category in enumerate(total_categories):
    transactions.loc[transactions.mcc_description.str.contains('|'.join(category)), 'category'] = i


In [99]:
transactions.sample(10)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,mcc_description,spending,income,category
21947,81827403,78 07:21:56,5533,1010,-8422.18,NaN,Автозапчасти и аксессуары,-8422.18,-0.0,3
2996,83063340,252 00:00:00,5541,1110,-23899.91,00593248,Станции техобслуживания,-23899.91,-0.0,3
19899,4181983,416 06:38:12,5541,1110,-22459.16,77023393,Станции техобслуживания,-22459.16,-0.0,3
15127,35680778,317 15:48:35,5541,1010,-39219.98,126927,Станции техобслуживания,-39219.98,-0.0,3
3626,94223809,214 00:00:00,5541,1110,-32943.09,20000958,Станции техобслуживания,-32943.09,-0.0,3
23477,74376012,308 11:52:21,7011,1010,-131386.07,726212,"Отели, мотели, базы отдыха, сервисы бронирования",-131386.07,-0.0,2
11579,48153184,340 11:15:46,5541,1010,-25199.17,10306224,Станции техобслуживания,-25199.17,-0.0,3
1653,93482866,50 00:00:00,5541,1110,-27929.98,NaN,Станции техобслуживания,-27929.98,-0.0,3
656,69161700,253 11:11:36,5541,1010,-22459.16,10141908,Станции техобслуживания,-22459.16,-0.0,3
10386,86467196,436 00:00:00,5541,1110,-39817.62,RU220031,Станции техобслуживания,-39817.62,-0.0,3


In [100]:
customer_transactions = transactions.groupby(['customer_id', 'category']).agg({'spending': 'sum', 'income': 'sum'})
customer_transactions.head()

spending  income
customer_id category                   
115617      3        -178222.85     0.0
127946      3         -71869.31     0.0
189257      2         -89836.63     0.0
            3         -65625.66     0.0
302489      3        -588284.61     0.0

In [101]:
from collections import defaultdict

# creating new dataframe, that contains only spending and income features for each category
customer_transactions_detailed = defaultdict(dict)

for row in customer_transactions.iterrows():
    customer_id, category = row[0]
    customer_transactions_detailed[customer_id][f'income_{category}'] = row[1]['income']
    customer_transactions_detailed[customer_id][f'spending_{category}'] = row[1]['spending']

In [102]:
customer_transactions_detailed = pd.DataFrame(customer_transactions_detailed).T

In [103]:
customer_transactions_detailed.head()


,income_3,spending_3,income_2,spending_2,income_4,spending_4,income_1,spending_1,income_5,spending_5
115617,0.00,-178222.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127946,0.00,-71869.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189257,0.00,-65625.66,0.0,-89836.63,NaN,NaN,NaN,NaN,NaN,NaN
302489,0.00,-588284.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361458,29196.91,-2609259.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# customers, that we have in customer_transactions_detailed have at least one transaction in one of the categories
# because we inner-joined transactions and tr_mcc_codes

all_customers_ids = set(gender_train.customer_id)
customers_with_transactions_ids = set(customer_transactions_detailed.index)
without = len(all_customers_ids.difference(customers_with_transactions_ids))
print(f'Customers without transactions: {without}')
print(f'Customers with transactions: {len(customers_with_transactions_ids)}')

Customers without transactions: 7784
Customers with transactions: 1078


In [108]:
# get customer with max non-zero and non-nan values
def non_zero_non_none(row):
    return len(row[row != 0].dropna())

customer_transactions_detailed['non_zero'] = customer_transactions_detailed.apply(non_zero_non_none, axis=1)
max_val = customer_transactions_detailed.sort_values('non_zero', ascending=False).iloc[0]['non_zero']
customer_transactions_detailed[customer_transactions_detailed.non_zero == max_val]

,income_3,spending_3,income_2,spending_2,income_4,spending_4,income_1,spending_1,income_5,spending_5,non_zero
2368868,0.00,-434728.44,281334.19,-10644156.21,NaN,NaN,NaN,NaN,NaN,NaN,4
3765322,0.00,-4244.78,0.00,-151934.63,0.0,-340031.65,NaN,NaN,NaN,NaN,4
7176153,33688.74,-735791.90,0.00,-188656.92,NaN,NaN,NaN,NaN,NaN,NaN,4
9497605,0.00,-3765967.73,0.00,-3977542.67,0.0,-108253.14,NaN,NaN,NaN,NaN,4
15104152,0.00,-809775.09,0.00,-16395.19,NaN,NaN,NaN,NaN,0.00,-4793014.91,4
16321752,0.00,-209484.44,NaN,NaN,NaN,NaN,NaN,NaN,125715.13,-167244.81,4
17389182,0.00,-293284.48,0.00,-28739739.92,0.0,-400578.17,NaN,NaN,NaN,NaN,4
17923253,21467.36,-1810280.60,0.00,-860747.21,NaN,NaN,NaN,NaN,NaN,NaN,4
19950741,44918.32,-3517759.43,0.00,-6422427.27,NaN,NaN,NaN,NaN,NaN,NaN,4
20085540,39550.58,-1846176.45,0.00,-5100644.07,NaN,NaN,NaN,NaN,NaN,NaN,4


## Вопросы к заданию 1 (6 баллов)

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. Существуют ли пользователи, у которых значения всех признаков не определены?
3. Выведите id пользователя, у которого самое большое количество непустых признаков.

# Задание 2 (3 балла)

Модель - многослойная нейронная сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Настроить параметры модели.

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены примерные, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

## Ответы:

# Задание 3 (2 балла)


Проверить качество модели не менее, чем на трёх разных метриках. 

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

## Ответы:

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [109]:
current_datetime = datetime.now()
print(current_datetime)

2022-12-12 13:09:07.771862
